<h1> 2c. Monitoring training </h1>

This notebook is Lab2c of CPB 102, Google's course on Machine Learning using Cloud ML.

In this notebook, we demonstrate monitoring the training of a TensorFlow model and visualizing the graph using TensorBoard. 

In [5]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

Code here is same as Lab2a.

In [6]:
def read_dataset(filename):
  return pd.read_csv(filename, header=None, names=['pickuplon','pickuplat','dropofflon','dropofflat','passengers','fare_amount'])

df_train = read_dataset('../lab1a/taxi-train.csv')
df_valid = read_dataset('../lab1a/taxi-valid.csv')
df_test = read_dataset('../lab1a/taxi-test.csv')
df_train[:5]
FEATURE_COLS = np.arange(0,5)
TARGET_COL   = 'fare_amount'

<h2> Monitoring tf.learn </h2>

Set up logging, monitoring and early stopping.

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    df_valid.iloc[:,FEATURE_COLS].values,
    df_valid[TARGET_COL].values,
    every_n_steps=50,
    metrics=["loss"],  # see slide notes for how to add other metrics
    early_stopping_metric="loss")

Note the checkpointing parameter to LinearRegressor; increase the number of steps (to something like 10,000) to see the early stopping kick in.

In [8]:
predictors = df_train.iloc[:,FEATURE_COLS].values # np.ndarray
targets = df_train[TARGET_COL].values
features = tf.contrib.learn.infer_real_valued_columns_from_input(predictors)
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.LinearRegressor(feature_columns=features,
                                         model_dir='taxi_model',
                    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1))

model = model.fit(predictors, targets, steps=1000)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float32)
INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 211.869
INFO:tensorflow:Step 101: loss = 86.5553
INFO:tensorflow:Step 201: loss = 86.5521
INFO:tensorflow:Saving checkpoints for 300 into taxi_model/model.ckpt.
INFO:tensorflow:Step 301: loss = 86.5501
INFO:tensorflow:Step 401: loss = 86.5485
INFO:tensorflow:Step 501: loss = 86.5471
INFO:tensorflow:Saving checkpoints for 600 into taxi_model/model.ckpt.
INFO:tensorflow:Step 601: loss = 86.546
INFO:tensorflow:Step 701: loss = 86.5451
INFO:tensorflow:Step 801: loss = 86.5441
INFO:tensorflow:Saving checkpoints for 900 into taxi_model/model.ckpt.
INFO:tensorflow:Step 901: loss = 86.5432
INFO:tensorflow:Saving checkpoints for 1000 into taxi_model/model.ckpt.
INFO:tensorflow:Loss for final step: 86.5425.


Can use tensorboard to look at the training.

In [9]:
%tensorboard start --logdir=taxi_model

In [12]:
%tensorboard stop --pid=4821

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License